In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.linear_model import LogisticRegression

In [3]:
bank = pd.read_csv('datasets/bank_data_processed.csv')
bank.head()

,Age,Income,Family,Education,Mortgage,Securities Account,CD Account,Online,CreditCard
0,34,180,1,3,0,0,0,0,0
1,38,130,4,3,134,0,0,0,0
2,46,193,2,3,0,0,0,0,0
3,38,119,1,2,0,0,1,1,1
4,42,141,3,3,0,1,1,1,0


In [4]:
X = bank.drop('CreditCard', axis= 1)
Y = bank['CreditCard']

In [5]:
#70%, 20%, 10%
x_train, x_hold_out, x_test = np.split(X, [int(.7*len(X)), int(.9*len(X))])
y_train, y_hold_out, y_test = np.split(Y, [int(.7*len(Y)), int(.9*len(Y))])

In [7]:
x_train.shape, x_hold_out.shape, x_test.shape

((336, 8), (96, 8), (48, 8))

In [9]:
clf1 = KNeighborsClassifier(n_neighbors= 10)
clf2 = RandomForestClassifier(n_estimators= 50)
clf3 = GaussianNB()

for clf in (clf1, clf2, clf3):
    clf.fit(x_train, y_train)

In [10]:
def get_predictions(x,y):
    pred_result = pd.DataFrame()
    
    i= 1
    for clf in (clf1, clf2, clf3):
        y_pred = clf.predict(x)
        print(clf.__class__.__name__, accuracy_score(y, y_pred))
        pred_result.insert(i-1, 'y_pred_' + str(i), y_pred)
        
        i += 1
    return pred_result

In [11]:
pred_results = get_predictions(x_hold_out, y_hold_out)

KNeighborsClassifier 0.7083333333333334
RandomForestClassifier 0.8229166666666666
GaussianNB 0.8229166666666666


In [14]:
pred_results.sample(10)

,y_pred_1,y_pred_2,y_pred_3
3,0,0,0
55,0,1,1
2,0,0,0
71,0,0,0
95,0,0,0
60,0,0,0
24,1,0,0
79,0,0,0
18,0,0,0
89,0,1,1


In [16]:
x_stack_train = pred_results
y_stack_train = y_hold_out

In [17]:
clf_stack = LogisticRegression(solver= 'lbfgs', C= 1, max_iter= 200)
clf_stack.fit(x_stack_train, y_stack_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
pred_result_test = get_predictions(x_test, y_test)

KNeighborsClassifier 0.6041666666666666
RandomForestClassifier 0.875
GaussianNB 0.8541666666666666


In [19]:
x_stack_test = pred_result_test
y_stack_pred = clf_stack.predict(x_stack_test)

In [20]:
accuracy_score(y_stack_pred, y_test)

0.8541666666666666